<a href="https://colab.research.google.com/github/wmezadev/CSE-450-TEAM-4/blob/neuralNetworkDraft/draft/BikeNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Welcome Bikes Neural Netwrok Model

lets instantiate the data and include librarys

In [140]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [141]:

bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

bikes.head()

,dteday,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c,casual,registered
0,1/1/11,1,0,0,0,1,0.81,0,3.28,3.0014,3,13
1,1/1/11,1,1,0,0,1,0.80,0,2.34,1.9982,8,32
2,1/1/11,1,2,0,0,1,0.80,0,2.34,1.9982,5,27
3,1/1/11,1,3,0,0,1,0.75,0,3.28,3.0014,3,10
4,1/1/11,1,4,0,0,1,0.75,0,3.28,3.0014,0,1


handling null values/objects that cannot be scaled

In [142]:
#you could just drop this column of date, but i believe its relevant.
#were gonna split it into one column with the day of year instead.
bikes['dteday'] = pd.to_datetime(bikes['dteday'])
bikes['day_of_year'] = bikes['dteday'].dt.dayofyear


#we dont need dteday anymore since its data has been split.
bikes = bikes.drop('dteday', axis=1)

#creating a column for total users (will be the target value)
totalUsers = bikes['casual'] + bikes['registered']
bikes['totalUsers'] = totalUsers

bikes.head()

,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c,casual,registered,day_of_year,totalUsers
0,1,0,0,0,1,0.81,0,3.28,3.0014,3,13,1,16
1,1,1,0,0,1,0.80,0,2.34,1.9982,8,32,1,40
2,1,2,0,0,1,0.80,0,2.34,1.9982,5,27,1,32
3,1,3,0,0,1,0.75,0,3.28,3.0014,3,10,1,13
4,1,4,0,0,1,0.75,0,3.28,3.0014,0,1,1,1


for fun, lets looks at the minimum and maximum values to know what to expect.

In [143]:
bikes.max()


season            4.0
hr               23.0
holiday           1.0
workingday        1.0
weathersit        4.0
hum               1.0
windspeed        57.0
temp_c           39.0
feels_like_c     50.0
casual          367.0
registered      886.0
day_of_year     365.0
totalUsers      977.0
dtype: float64

In [144]:
bikes.min()

season           1.00
hr               0.00
holiday          0.00
workingday       0.00
weathersit       1.00
hum              0.00
windspeed        0.00
temp_c          -7.06
feels_like_c   -16.00
casual           0.00
registered       0.00
day_of_year      1.00
totalUsers       1.00
dtype: float64

Defining Features and Target values

In [145]:
features = bikes[['season', 'hr', 'holiday', 'workingday', 'weathersit', 'hum', 'windspeed', 'temp_c', 'feels_like_c', 'day_of_year']]  
#our target value is actually the TOTAL number of people that came by, thats the casual + registered users

target = bikes['totalUsers']  


Using a scaler on the values (they're all integers or floats so this is relatively simple)

In [146]:
scaled_features = MinMaxScaler().fit_transform(features)
scaled_features_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_features_df.tail()


,season,hr,holiday,workingday,weathersit,hum,windspeed,temp_c,feels_like_c,day_of_year
16632,1.0,0.826087,0.0,1.0,0.000000,0.66,0.000000,0.306122,0.3485,0.917582
16633,1.0,0.869565,0.0,1.0,0.000000,0.66,0.000000,0.306122,0.3485,0.917582
16634,1.0,0.913043,0.0,1.0,0.000000,0.75,0.105263,0.285714,0.3182,0.917582
16635,1.0,0.956522,0.0,1.0,0.000000,0.75,0.000000,0.285714,0.3333,0.917582
16636,1.0,1.000000,0.0,1.0,0.333333,0.75,0.105263,0.285714,0.3182,0.917582


splitting the data into training and testing *sets*

In [147]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)

Time to build the actual model

In [159]:
#we need the number of features for this.
num_features = scaled_features.shape[1]


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(80, activation='relu', input_shape=(num_features,)),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(40, activation='relu'),  
    tf.keras.layers.Dense(20, activation='relu'),  
    tf.keras.layers.Dense(1)  # Output layer
])

#compiling the model, using mean squared error since it works great for neural networks
#and the adam optimizer
model.compile(loss='mean_squared_error', optimizer='adam')


Lets train the model now!

In [161]:
model.fit(X_train, y_train, epochs=150, batch_size=14) #epoch 100 is a LOT, we can reduce this to avoid overfitting if we choose.

Epoch 1/150
951/951 [==============================] - 2s 2ms/step - loss: 6810.3076
Epoch 2/150
951/951 [==============================] - 2s 2ms/step - loss: 6493.0635
Epoch 3/150
951/951 [==============================] - 2s 2ms/step - loss: 6532.5039
Epoch 4/150
951/951 [==============================] - 3s 3ms/step - loss: 6343.6934
Epoch 5/150
951/951 [==============================] - 2s 2ms/step - loss: 6229.0298
Epoch 6/150
951/951 [==============================] - 2s 2ms/step - loss: 6384.0415
Epoch 7/150
951/951 [==============================] - 2s 2ms/step - loss: 6052.7158
Epoch 8/150
951/951 [==============================] - 2s 2ms/step - loss: 6067.9385
Epoch 9/150
951/951 [==============================] - 2s 2ms/step - loss: 5913.0957
Epoch 10/150
951/951 [==============================] - 3s 3ms/step - loss: 6007.5679
Epoch 11/150
951/951 [==============================] - 2s 2ms/step - loss: 5911.5913
Epoch 12/150
951/951 [==============================] - 2s 2ms/

Now to use our model to make predictions

In [150]:
loss = model.evaluate(X_test, y_test)
print('Mean Squared Error:', loss)
predictions = model.predict(X_test)

# Print predicted values
predicted_values = pd.DataFrame(predictions)
predicted_values.head()





104/104 [==============================] - 0s 2ms/step - loss: 4695.7988
Mean Squared Error: 4695.798828125
104/104 [==============================] - 0s 1ms/step


,0
0,5.500486
1,330.381470
2,236.307816
3,27.882105
4,130.423965
